In [27]:
# 셋팅하기 - 필요한 모듈/라이브러리
import os
import sys
import urllib.request
import datetime
import time
import json

In [28]:
# 네이버 개발자에서 생성한 개인 API 호출 아이디,비번 가져오기
# 원래 이건 config 파일에서 끌어와야함
client_id = ""
client_secret = ""

In [43]:
# [code 1]
def getRequestUrl(url):
    req = urllib.request.Request(url) #url 불러오기
    req.add_header("X-Naver-Client-Id",client_id) 
    # Naver api 사용 id를 검색 url header에 넣어줘야 함
    req.add_header("X-Naver-Client-Secret",client_secret) 
    
    # 예외처리문으로 구성해서 네이버 서버의 응답을 확인
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s]url Request Success" %datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e: #에러 예외처리
        print(e)
        print("[%s]Error for URL : %s" %(datetime.datetime.now(), url))
        return None

In [44]:
# [code 2]
def getNaverSearch(node, scrText, start, display):
    # 네이버 검색 API 정보로 URL을 구성 함
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json"%node
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(scrText),start,display)
    
    url = base + node + parameters
    
    # 만들어진 URL을 이용하여 getRequestUrl() 호출해서 받은 utf-8 디코드 응답을 responseDecode에 저장
    responseDecode = getRequestUrl(url)
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)
    # 서버에서 받은 JSON 형태의 응답객체를 파이썬 객체로 로드하여 반환

In [51]:
# [code 3]
# getPostData 함수를 정의해서 검색결과에서 필요한 데이터 항목만 추출해서 변수에 저장

# [CODE 3]
# getPostDate 함수 정의 => 검색결과에서 필요한 데이터 항목만 추출해서 변수에 저장
def getPostData(post, jsonResult, cnt): # cnt: 검색된 count 숫자
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    # 네이버에서 제공하는 시간형식을 필요한 형식으로 전환
    # Tue, 14 Feb 2017 18:46:00 +0900
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    # 딕셔너리 형태의 {키: 값}으로 구성해서 jsonResult로 구성
    jsonResult.append({'cnt': cnt, 'title': title, 'description': description, 
                       'org_link': org_link, 'link': link, 'pDate': pDate})
    return

In [52]:
# [code 4]
def main():
    node = 'news' # 크롤링할 대상
    srcText = input("검색어를 입력하세요: ")
    cnt = 0
    jsonResult = []
    
    jsonResponse = getNaverSearch(node, srcText, 1, 100) # [code 2]
    total = jsonResponse['total']
    
    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) # [code 3]
            
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) # [code 2]
        
    print('전체 검색 : %d 건'%total)
    
    with open('%s_naver_%s.json' %(srcText,node),'w',encoding='utf-8') as outfile:
        jsonFIle = json.dumps(jsonResult, indent=4, sort_keys=True,
                             ensure_ascii=False)
        outfile.write(jsonFIle)
    print("가져온 데이터 : %d건"%(cnt))
    print("%s_naver_%s.json SAVED" %(srcText, node))

In [54]:
# [code 5]
main()

검색어를 입력하세요: 오징어게임
[2021-10-26 12:02:38.108633]url Request Success
[2021-10-26 12:02:38.309596]url Request Success
[2021-10-26 12:02:38.533532]url Request Success
[2021-10-26 12:02:38.737412]url Request Success
[2021-10-26 12:02:38.958050]url Request Success
[2021-10-26 12:02:39.175531]url Request Success
[2021-10-26 12:02:39.383169]url Request Success
[2021-10-26 12:02:39.637383]url Request Success
[2021-10-26 12:02:39.868116]url Request Success
[2021-10-26 12:02:40.096300]url Request Success
HTTP Error 400: Bad Request
[2021-10-26 12:02:40.190269]Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%98%A4%EC%A7%95%EC%96%B4%EA%B2%8C%EC%9E%84&start=1001&display=100
전체 검색 : 24483 건
가져온 데이터 : 1000건
오징어게임_naver_news.json SAVED
